### Transfer Learning with resnet50

In [192]:
from torchvision import models
import numpy as np
import torch as th

from torch.utils.data import DataLoader
from tqdm import tqdm

from imgdata import DefaultTrainSet, DefaultTestSet

from torch import nn
import torch.optim as optim

In [193]:
%matplotlib inline

if th.cuda.is_available():
  # Make CuDNN Determinist
  th.backends.cudnn.deterministic = True

# Define default device, we should use the GPU (cuda) if available
device = th.device("cuda" if th.cuda.is_available() else "cpu")
device

device(type='cpu')

### Set constants

In [194]:
lr = 1e-4
epochs = 5
batch_size = 16
class_label = ['faces', 'dog', 'airplanes', 'keyboard', 'cars']

In [195]:
train_set = DefaultTrainSet()

train_dataloader = th.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

### Define model

In [196]:
resnet50 = models.resnet50(pretrained=True)
fc_inputs = resnet50.fc.in_features

In [197]:
resnet50.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 5)
)

In [198]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet50.parameters(), lr=lr)

### Train model

In [199]:
def train(_net, n_epochs, train_loader, _optimizer, _criterion):
    """
    Train a neural network and print statistics of the training

    :param _criterion:
    :param _optimizer:
    :param train_loader: loader for the training data
    :param _net: (PyTorch Neural Network)
    :param n_epochs: (int)  Number of iterations on the training set
    """
    print("---------- HYPERPARAMETERS ----------")
    print("n_epochs=", n_epochs)
    print("-" * 40)
    accuracies = []
    losses = []
    # Move model to gpu if possible
    _net = _net.to(device)
    for epoch_count, epoch in enumerate(range(n_epochs)):
        # Set the model to training mode
        _net.train()
        # Initialize the variables to compute the average loss
        total_loss = 0.00
        total_correct = 0
        pbar = tqdm(train_loader)
        # Iterate over the training set
        for i, data in enumerate(pbar):

            # Get the inputs
            inputs, labels = data['imNorm'], data['label']
            # Move them to the correct device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            _optimizer.zero_grad()

            # Forward pass
            outputs = _net(inputs)
             # Calculate Loss: softmax --> cross entropy loss
            loss = _criterion(outputs, labels)
            _, preds = th.max(outputs, 1)
            # Getting gradients w.r.t. parameters
            loss.backward()

            # Updating parameters
            _optimizer.step()

            # Update the running variables
            total_loss += loss.item()
            avg_loss = total_loss / (i + 1)
            total_correct += th.sum(preds == labels.data)
            pbar.set_description("Epoch: {}/{}".format(epoch_count + 1, n_epochs))
            pbar.set_postfix(avergae_loss=avg_loss, running_accuracy=th.sum(preds == labels).item() / float(labels.size(0)))
            # save the model
            model_name = f"models/resNet50.pth"
            th.save(_net.state_dict(), model_name)

        # Calculate the average loss and accuracy of the epoch
        losses.append(total_loss/len(train_loader))
        accuracies.append((total_correct/train_loader.sampler.num_samples).item())
        # print losses and accuracies in the last epoch
        if epoch_count == n_epochs - 1:
            print("---------- FINAL RESULTS ----------")
            print("Losses fot every epoch:", losses)
            print("Average accuracy for each epoch: {}".format(accuracies))
            print("-" * 40)
    return losses, accuracies

In [200]:
losses, accuracies = train(resnet50, epochs, train_dataloader, optimizer, loss_func)

---------- HYPERPARAMETERS ----------
n_epochs= 5
----------------------------------------


Epoch: 5/5: 100%|██████████| 22/22 [02:26<00:00,  6.65s/it, avergae_loss=0.0591, running_accuracy=0.929]

---------- FINAL RESULTS ----------
Losses fot every epoch: [0.7745353762399066, 0.09345721411095424, 0.07319857798178088, 0.06157978723587638, 0.059090126157653605]
Average accuracy for each epoch: [0.854285717010498, 0.9828571677207947, 0.9885714054107666, 0.9885714054107666, 0.9885714054107666]
----------------------------------------


### Evaluate model

In [201]:
# load test set and create dataloader
test_set = DefaultTestSet()
test_loader = DataLoader(test_set)
test_y = [i['label'] for i in test_set]

In [204]:
def predict(_model, _tests):
  predicts = []
  with th.no_grad():
    for i in tqdm(_tests):
        _model.eval()
        images = i['imNorm'].clone().to(device)
        outputs = _model(images)
        _, predicted = th.max(outputs.data, 1)
        predicts.append(predicted.item())
  return predicts

resnet50.load_state_dict(th.load('models/resNet50.pth'))
predict_y = predict(resnet50, test_loader)

100%|██████████| 50/50 [00:06<00:00,  7.83it/s]


In [205]:
correct = []
incorrect = []
for i, j in enumerate(predict_y):
    if j == test_y[i]:
      correct.append(i)
    else:
      incorrect.append(i)

# generate confusion matrix
def confusion_matrix(y_true, y_pred):
  n_class = np.unique(y_true).shape[0]
  cm = np.zeros((n_class, n_class))
  for i in range(len(y_true)):
    cm[y_true[i], y_pred[i]] += 1
  return cm


cm = confusion_matrix(test_y, predict_y)
print(cm)

def print_accuracy(cm):
  for i in range(5):
    print(f'the accuracy by class {class_label[i]} : {cm[i, i] / np.sum(cm[i, :])}')
  # overall accuracy
  print(f'overall accuracy : {np.sum(cm[range(5), range(5)]) / np.sum(cm)}')

print_accuracy(cm)

[[10.  0.  0.  0.  0.]
 [ 0. 10.  0.  0.  0.]
 [ 0.  0. 10.  0.  0.]
 [ 0.  0.  1.  9.  0.]
 [ 0.  0.  0.  0. 10.]]
the accuracy by class faces : 1.0
the accuracy by class dog : 1.0
the accuracy by class airplanes : 1.0
the accuracy by class keyboard : 0.9
the accuracy by class cars : 1.0
overall accuracy : 0.98
